<a href="https://colab.research.google.com/github/roklp/MLP34/blob/main/240312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/data/머신러닝/data/real_final.csv")

# 데이터의 처음 몇 행 확인
print(data.head())


       매출_금액  기준_년도  기준_분기    시간대 상권_구분_코드_명            상권_코드_명    당월_매출_금액  \
0   62575582   2021      1  00~06       골목상권  포이초등학교(개포목련어린이공원)   770055585   
1  190893636   2021      1  00~06       발달상권               선정릉역  2834665057   
2   84038340   2021      1  00~06       골목상권              언북중학교   698553091   
3  180614316   2021      1  00~06       발달상권             개나리아파트  2360145485   
4  191954664   2021      1  00~06       골목상권             대청초등학교  2033028774   

   당월_매출_건수    주중_매출_금액   주말_매출_금액  ...  대학교_수  백화점_수  슈퍼마켓_수  극장_수  숙박_시설_수  \
0    105130   562210834  207844751  ...    0.0    0.0     0.0   0.0      0.0   
1    397588  2250471191  584193866  ...    1.0    0.0     0.0   0.0      0.0   
2     92578   531900150  166652941  ...    4.0    0.0     0.0   0.0      0.0   
3    343200  1862929139  497216346  ...    0.0    0.0     0.0   0.0      0.0   
4    316583  1389093802  643934972  ...    0.0    0.0     0.0   0.0      0.0   

   공항_수  철도_역_수  버스_터미널_수  지하철_역_수  버스_정거장_수

In [15]:
print(data.columns[:50])


Index(['매출_금액', '기준_년도', '기준_분기', '시간대', '상권_구분_코드_명', '상권_코드_명', '당월_매출_금액',
       '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액',
       '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액',
       '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액',
       '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수',
       '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수',
       '일요일_매출_건수', '시간대_건수~06_매출_건수', '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수',
       '시간대_건수~17_매출_건수', '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '연령대_10_매출_건수',
       '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수',
       '연령대_60_이상_매출_건수', '엑스좌표_값', '와이좌표_값', '행정동_코드_명', '영역_면적', '총_유동인구_수',
       '연령대_10_유동인구_수'],
      dtype='object')


In [37]:
# 결측치가 있는 열에 대해 평균값으로 대체
data.fillna(data.mean(), inplace=True)


In [32]:
# 데이터프레임 정보 확인
print(data.info())

# 모든 열을 숫자형으로 변환하고 결측값을 평균값으로 채우기
for column in data.columns:
    if data[column].dtype == 'object':  # 숫자형이 아닌 열만 처리
        try:
            data[column] = pd.to_numeric(data[column], errors='coerce')  # 숫자형으로 변환
            mean_value = data[column].mean()  # 평균값 계산
            data[column].fillna(mean_value, inplace=True)  # 결측값 평균값으로 채우기
        except ValueError:
            pass  # 숫자형으로 변환할 수 없는 경우 처리하지 않음

# 모든 열이 숫자형인지 확인
print(data.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3388 entries, 0 to 3599
Columns: 115 entries, 매출_금액 to 버스_정거장_수
dtypes: float64(31), int64(80), object(4)
memory usage: 3.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3388 entries, 0 to 3599
Columns: 115 entries, 매출_금액 to 버스_정거장_수
dtypes: float64(35), int64(80)
memory usage: 3.0 MB
None


In [ ]:
import pandas as pd

# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/data/머신러닝/data/real_final.csv")

# 이상치를 확인하고 제거하는 함수
def remove_outliers(df, column):
    # IQR 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # 범위 계산
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 제거
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return df

# 결측치를 중앙값으로 대체
def impute_missing_values(df, column):
    # 중앙값으로 결측치 대체
    median_value = df[column].median()
    df[column].fillna(median_value, inplace=True)

    return df

# 이상치 처리
data = remove_outliers(data, '매출_금액')

# 결측치 처리
data = impute_missing_values(data, '매출_금액')

print("이상치와 결측치 처리 후 데이터:")
print(data.head())


In [ ]:
from scipy.stats import chi2_contingency

# 전체 변수들의 독립성 검정
for column1 in data.columns:
    for column2 in data.columns:
        # 같은 변수에 대한 검정이거나, '매출_금액'을 포함한 조합이 아닌 경우 스킵
        if column1 == column2 or '매출_금액' not in [column1, column2]:
            continue

        # 교차표 생성
        crosstab = pd.crosstab(data[column1], data[column2])

        # 카이제곱 검정 수행
        chi2, p_value, _, _ = chi2_contingency(crosstab)

        # 결과 출력
        print(f"카이제곱 검정 결과 ({column1} vs. {column2}):")
        print(f"Chi-square: {chi2}")
        print(f"P-value: {p_value}")
        alpha = 0.05
        if p_value < alpha:
            print("귀무가설을 기각하고, 두 변수 간에는 통계적으로 유의한 관계가 있습니다.")
        else:
            print("귀무가설을 기각할 수 없으며, 두 변수 간에는 통계적으로 유의한 관계가 없습니다.")
        print()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 상관 행렬 계산
correlation_matrix = data.corr()

# 상관 행렬 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()


In [38]:
import numpy as np

# 무한대 값을 NaN으로 대체
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 해당 열의 평균값으로 대체
data.fillna(data.mean(), inplace=True)

# 회귀 분석 다시 시도
model = sm.OLS(dependent_var, independent_vars).fit()
print(model.summary())


MissingDataError: exog contains inf or nans

In [ ]:
print(data.describe())


In [ ]:
# 평균 계산
mean_values = data.mean()

print("전체적인 평균:")
print(mean_values)


In [ ]:
# 분산 계산
variance_values = data.var()

print("분산:")
print(variance_values)


In [ ]:
# 표준편차 계산
std_deviation_values = data.std()

print("표준편차:")
print(std_deviation_values)


In [ ]:
# 중앙값 계산
median_values = data.median()

print("중앙값:")
print(median_values)


In [ ]:
# 매출_금액 열에 대한 기술통계 계산
sales_stats = data['매출_금액'].describe()

# 분산 계산
variance = data['매출_금액'].var()

# 표준편차 계산
std_deviation = data['매출_금액'].std()

# 중앙값 계산
median = data['매출_금액'].median()

# 결과 출력
print("매출_금액 열에 대한 기술통계:")
print(sales_stats)
print("\n분산:", variance)
print("표준편차:", std_deviation)
print("중앙값:", median)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(data['매출_금액'], bins=30, color='skyblue', edgecolor='black')
plt.title('매출_금액 분포', fontsize=15)
plt.xlabel('매출_금액', fontsize=12)
plt.ylabel('빈도', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.boxplot(data=data, y='매출_금액', color='lightblue')
plt.title('매출_금액 분포 - Box Plot', fontsize=15)
plt.ylabel('매출_금액', fontsize=12)
plt.grid(True)
plt.show()
